# Best joint account accounts to use
To access the notebook and codes, you can access it through this [repo](https://github.com/zhyoung17/joint-account).

This repo will analyse different joint accounts to use for various scenarios. In this ipynb file, I will examine and use the most ideal joint account for fresh graduates who are earning around the median starting salary of ~ S$4,000. 

Please note that this is NOT financial advice, but instead just a data analyst who is in the same scenario. Use the bank accounts at your own discretion and not mine. Enjoy!

## Why use a joint account?
As a couple, we need to aim to maximise our cash in a low-risk environment to ensure that we are able to use it for our own future needs. One example is to use the cash for BTO, where cash might be needed for renovations and furnishing. 

Why not use our own accounts? This is up to personal preference. Personally, it gives a sense of financial ownership where couples are able to pay off shared expenses through a shared account. Moreover, the joint account ensures that the money is transparent for both parties to access too, so less bickering about who-pays-what.

Lastly, you might be wondering if it is better off to invest the money into an index fund or SPY ETF to maximise your interest. Yes! The SPY provides a steady YOY return as compared to a savings fund (11.98% vs 2.80%). But of course you must understand that a joint account is not an investment account. The purpose of a joint account is to ensure that the money within is liquid and accessible to both parties. These criterias are not met in an ETF or index fund.

# Accounts
We will be examining the following savings accounts:
1. UOB One
2. UOB Stash
3. OCBC 360

Main assumptions:
1. Consistent monthly contribution: Both parties contribute their respective share diligently.
2. Consistent monthly growth: If you intend to use the money for expenses (meals, pets, furnishing), I expect the money to be returned to the account ASAP. This is extremely important as the goal of the joint account is to FORCE savings.

## Vanilla method: No interest, just vibes
This will be the benchmark method, where we solely put the money inside a bank account that does not accumulate interest.

In [31]:
def no_interest(person1_contribution, person2_contribution, initial_amount = 0, years = 3):
    months = years * 12
    total_contribution = person1_contribution + person2_contribution
    value = initial_amount + (total_contribution * months)
    return value

## [UOB One](https://www.uob.com.sg/personal/save/everyday-accounts/one-account.page)
This one is slightly tricky, as it will require one party to credit the income into the account, and also spend at least $500 using the respective UOB card to attain the max interest rate.

![uob_one](image/uob_one.png)

In this scenario, I am assuming that the third scenario, spending minimum $500 using a valid UOB card and crediting your salary, is attained. This is the strategy that I would recommend all couples to acheive.

In theory, the UOB cards are one of the most flexible and generous cards in the market, and this might impact your decision to use the card.

Below is the code that will be used to calculate the interest and returns of the UOB one card.

In [32]:
def uob_one(person1_contribution, person2_contribution, account_balance=0, years=3):
    current_value_of_account = account_balance # Start with the initial balance

    if years == 0:
        return current_value_of_account

    total_monthly_contribution = person1_contribution + person2_contribution
    num_months = years * 12

    # UOB One Scenario 3 interest rate tiers (these are ANNUAL rates)
    tiers_annual_rates = [
        {"limit": 75000, "rate": 0.0230},    # For the first S$75,000
        {"limit": 50000, "rate": 0.0380},    # For the next S$50,000
        {"limit": 25000, "rate": 0.0530},    # For the next S$25,000
        {"limit": float('inf'), "rate": 0.0005} # For amounts above S$150,000
    ]

    for _ in range(num_months):
        # 1. Add monthly contributions (assuming at the start of the month)
        current_value_of_account += total_monthly_contribution
        
        # 2. Calculate monthly interest on the new current_value_of_account
        # This current_value_of_account is treated as the MAB for this month's interest calculation.
        balance_for_interest_calc = current_value_of_account
        monthly_interest_earned_this_month = 0.0
        
        remaining_balance_for_tiering = balance_for_interest_calc

        if remaining_balance_for_tiering > 0: # Only calculate interest if there's a positive balance
            for tier in tiers_annual_rates:
                if remaining_balance_for_tiering <= 0:
                    break 

                annual_rate = tier["rate"]
                monthly_rate = annual_rate / 12 # Convert annual tier rate to monthly
                
                tier_band_size = tier["limit"] # This is the size of the current tier's band

                amount_in_this_tier = 0
                if tier_band_size == float('inf'): 
                    # This is the highest tier (e.g., "Above S$150,000")
                    amount_in_this_tier = remaining_balance_for_tiering
                else:
                    amount_in_this_tier = min(remaining_balance_for_tiering, tier_band_size)
                
                interest_for_this_tier_portion = amount_in_this_tier * monthly_rate
                monthly_interest_earned_this_month += interest_for_this_tier_portion
                
                remaining_balance_for_tiering -= amount_in_this_tier # Reduce balance for next tier
                
                if tier_band_size == float('inf'): # All balance has been processed by this tier
                    break
        
        # 3. Add earned monthly interest to the account value
        current_value_of_account += monthly_interest_earned_this_month
            
    return current_value_of_account

## [UOB Stash](https://www.uob.com.sg/personal/save/savings-accounts/stash-account.page)
This is a foolproof method to save money. To earn the bonus interest, you are required to:
1. Have more than $10,000 in your bank account
2. Maintain or increase the account balance from last month's balance.

So based on the initial assumption set, which is to not use the money from the joint account, the bonus interest will be set in place. 

![uob_stash](image/uob_stash.png)

Some couples might enjoy using this account as the money inside is basically locked like a vault, meaning that there is no obligation to spend any money to unlock the maximum interest rate.

In [33]:
def uob_stash(person1_monthly_contribution, person2_monthly_contribution, initial_account_balance=0, years=3):

    current_value_of_account = initial_account_balance  # Start with the initial balance

    if years == 0:
        return current_value_of_account

    total_monthly_contribution = person1_monthly_contribution + person2_monthly_contribution
    num_months = years * 12

    # UOB Stash Account interest rate tiers (ANNUAL rates from "Total Interest (p.a.)")
    stash_tiers_annual_rates = [
        {"limit": 10000, "rate": 0.0005},   # First S$10,000 @ 0.05%
        {"limit": 30000, "rate": 0.0200},   # Next S$30,000 @ 2.00% (up to $40k)
        {"limit": 30000, "rate": 0.0300},   # Next S$30,000 @ 3.00% (up to $70k)
        {"limit": 30000, "rate": 0.0500},   # Next S$30,000 @ 5.00% (up to $100k)
        {"limit": float('inf'), "rate": 0.0005} # Above S$100,000 @ 0.05%
    ]

    for _ in range(num_months):
        # 1. Add monthly contributions (assuming at the start of the month)
        current_value_of_account += total_monthly_contribution
        
        # 2. Calculate monthly interest on the new current_value_of_account
        balance_for_interest_calc = current_value_of_account
        monthly_interest_earned_this_month = 0.0
        
        remaining_balance_for_tiering = balance_for_interest_calc

        if remaining_balance_for_tiering > 0: # Only calculate interest if there's a positive balance
            for tier in stash_tiers_annual_rates:
                if remaining_balance_for_tiering <= 0:
                    break 

                annual_rate = tier["rate"]
                monthly_rate = annual_rate / 12 # Convert annual tier rate to monthly
                
                tier_band_size = tier["limit"] # This is the size of the current tier's band

                amount_in_this_tier = 0
                if tier_band_size == float('inf'): 
                    amount_in_this_tier = remaining_balance_for_tiering
                else:
                    amount_in_this_tier = min(remaining_balance_for_tiering, tier_band_size)
                
                interest_for_this_tier_portion = amount_in_this_tier * monthly_rate
                monthly_interest_earned_this_month += interest_for_this_tier_portion
                
                remaining_balance_for_tiering -= amount_in_this_tier # Reduce balance for next tier
                
                if tier_band_size == float('inf'): # All balance has been processed
                    break
        
        # 3. Add earned monthly interest to the account value
        current_value_of_account += monthly_interest_earned_this_month
            
    return current_value_of_account

## [OCBC 360](https://www.ocbc.com/personal-banking/deposits/360-savings-account)
Last but certainly not least is the OCBC 360. OCBC 360 has a much more "thorough" plan, meaning that there is certainly a better chance to maximise your interest rates. But let's be real, it is just a marketing ploy. 

![ocbc_360](image/ocbc_360.png)

Realistically, you will only hit the first three categories: salary, save, and spend. I think there is not much purpose insuring or investing through OCBC - Insurance companies and index funds are much more catered towards the respective functions.

On a side note, I am not a fan of investing your money through insurance companies (sorry FAs), there are more low-risk + high-returns + high-liquidity methods in the market.

Side-side note, OCBC cards are not extremely popular in the credit card scene (correct me if I am wrong), so I added a switch in the function below to toggle it off.

In [34]:
def ocbc360(
    person1_monthly_contribution, 
    person2_monthly_contribution, 
    initial_account_balance=0, 
    years=3,
    salary_condition_met=True, # True if salary >= S$1,800 credited
    save_condition_met=True,   # True if ADB increased by >= S$500 monthly
    spend_condition_met=True   # True if card spend >= S$500
    # Ok, you will most likely meet all 3 conditions if you use an OCBC card
):
    # Input validation
    if not all(isinstance(val, (int, float)) and val >= 0 for val in [person1_monthly_contribution, person2_monthly_contribution, initial_account_balance]):
        raise ValueError("Contributions and initial account balance must be non-negative numbers.")
    if not isinstance(years, int) or years < 0:
        raise ValueError("Years must be a non-negative integer.")
    if not all(isinstance(val, bool) for val in [salary_condition_met, save_condition_met, spend_condition_met]):
        raise ValueError("Condition flags (salary_condition_met, save_condition_met, spend_condition_met) must be boolean.")

    current_value_of_account = initial_account_balance

    if years == 0:
        return current_value_of_account

    total_monthly_contribution = person1_monthly_contribution + person2_monthly_contribution
    num_months = years * 12

    # --- Define Annual Interest Rate Components (p.a.) ---
    base_rate_pa = 0.0005  # Assumed 0.05% p.a.

    # Bonus rates from conditions
    salary_bonus_rates_pa = {"tier1": 0.0160, "tier2": 0.0320} # First 75k, Next 25k
    save_bonus_rates_pa = {"tier1": 0.0060, "tier2": 0.0120}   # First 75k, Next 25k
    spend_bonus_rates_pa = {"tier1": 0.0050, "tier2": 0.0050} # Assumed 0.50% on first 100k

    # --- Calculate effective monthly rates for each balance tier ---
    # Tier 1: First S$75,000
    rate_tier1_annual = base_rate_pa
    if salary_condition_met:
        rate_tier1_annual += salary_bonus_rates_pa["tier1"]
    if save_condition_met:
        rate_tier1_annual += save_bonus_rates_pa["tier1"]
    if spend_condition_met:
        rate_tier1_annual += spend_bonus_rates_pa["tier1"]
    rate_tier1_monthly = rate_tier1_annual / 12

    # Tier 2: Next S$25,000 (balances from S$75,000.01 to S$100,000)
    rate_tier2_annual = base_rate_pa
    if salary_condition_met:
        rate_tier2_annual += salary_bonus_rates_pa["tier2"]
    if save_condition_met:
        rate_tier2_annual += save_bonus_rates_pa["tier2"]
    if spend_condition_met:
        rate_tier2_annual += spend_bonus_rates_pa["tier2"]
    rate_tier2_monthly = rate_tier2_annual / 12

    # Tier 3: Balances Above S$100,000
    rate_tier3_annual = base_rate_pa  # Only base rate applies
    rate_tier3_monthly = rate_tier3_annual / 12
    
    # --- Monthly Simulation Loop ---
    for _ in range(num_months):
        current_value_of_account += total_monthly_contribution
        
        balance_for_interest_calc = current_value_of_account
        monthly_interest_earned = 0.0
        
        if balance_for_interest_calc <= 0:
            # No interest if balance is zero or negative (though contributions might make it positive)
            # current_value_of_account might have become positive due to contributions
            if current_value_of_account <=0: continue


        # Tier 1 Calculation (First S$75,000)
        amount_on_tier1 = min(balance_for_interest_calc, 75000.0)
        monthly_interest_earned += amount_on_tier1 * rate_tier1_monthly
        remaining_balance_after_tier1 = balance_for_interest_calc - amount_on_tier1

        # Tier 2 Calculation (Next S$25,000)
        if remaining_balance_after_tier1 > 0:
            amount_on_tier2 = min(remaining_balance_after_tier1, 25000.0)
            monthly_interest_earned += amount_on_tier2 * rate_tier2_monthly
            remaining_balance_after_tier2 = remaining_balance_after_tier1 - amount_on_tier2

            # Tier 3 Calculation (Above S$100,000)
            if remaining_balance_after_tier2 > 0:
                amount_on_tier3 = remaining_balance_after_tier2
                monthly_interest_earned += amount_on_tier3 * rate_tier3_monthly
        
        current_value_of_account += monthly_interest_earned
            
    return current_value_of_account

# Scenarios
Now we will break this down into 3 simple scenarios, with the focus being on fresh grad couples who are looking to create their joint account. 

## Scenario 1: Initial balance of \$10,000, with a monthly contribution of \$1,000/person
This is the most common and conservative method to maximise your joint cash. Assuming that each person earns $3,800 after CPF deduction, this will result in each person having $2,800 for their own spendings and investment - comfortable if you are not paying off any big ticket items such as a car or renovation

In [35]:
person1_contribution = 1000.0
person2_contribution = 1000.0
account_balance = 10000.0
years = 3
no_interest_value = no_interest(person1_contribution, person2_contribution, account_balance, years)
print(f"No Interest Account Value after {years} years: S${no_interest_value:.2f}")
uob_one_value = uob_one(person1_contribution, person2_contribution, account_balance, years)
print(f"UOB One Account Value after {years} years: S${uob_one_value:.2f}")
uob_stash_value = uob_stash(person1_contribution, person2_contribution, account_balance, years)
print(f"UOB Stash Account Value after {years} years: S${uob_stash_value:.2f}")
ocbc360_value = ocbc360(person1_contribution, person2_contribution, account_balance, years)
print(f"OCBC 360 Account Value after {years} years: S${ocbc360_value:.2f}")



No Interest Account Value after 3 years: S$82000.00
UOB One Account Value after 3 years: S$85361.40
UOB Stash Account Value after 3 years: S$84787.06
OCBC 360 Account Value after 3 years: S$86053.90


## Scenario 2: Initial balance of \$0, with a monthly contribution of \$1,000/person
In this scenario, we assume that both have no money in their savings account and decided to open an account from scratch. Some might relate to this as they wish to maximise their own personal investments and savings.


In [36]:
person1_contribution = 1000.0
person2_contribution = 1000.0
account_balance = 0
years = 3
no_interest_value = no_interest(person1_contribution, person2_contribution, account_balance, years)
print(f"No Interest Account Value after {years} years: S${no_interest_value:.2f}")
uob_one_value = uob_one(person1_contribution, person2_contribution, account_balance, years)
print(f"UOB One Account Value after {years} years: S${uob_one_value:.2f}")
uob_stash_value = uob_stash(person1_contribution, person2_contribution, account_balance, years)
print(f"UOB Stash Account Value after {years} years: S${uob_stash_value:.2f}")
ocbc360_value = ocbc360(person1_contribution, person2_contribution, account_balance, years)
print(f"OCBC 360 Account Value after {years} years: S${ocbc360_value:.2f}")

No Interest Account Value after 3 years: S$72000.00
UOB One Account Value after 3 years: S$74611.03
UOB Stash Account Value after 3 years: S$73947.47
OCBC 360 Account Value after 3 years: S$75135.73


## Scenario 3: Initial balance of \$0, with a monthly contribution of \$2,000/person
Similar to above, but slightly more aggressive in the way you are contributing into the account

In [37]:
person1_contribution = 2000.0
person2_contribution = 2000.0
account_balance = 0
years = 3
no_interest_value = no_interest(person1_contribution, person2_contribution, account_balance, years)
print(f"No Interest Account Value after {years} years: S${no_interest_value:.2f}")
uob_one_value = uob_one(person1_contribution, person2_contribution, account_balance, years)
print(f"UOB One Account Value after {years} years: S${uob_one_value:.2f}")
uob_stash_value = uob_stash(person1_contribution, person2_contribution, account_balance, years)
print(f"UOB Stash Account Value after {years} years: S${uob_stash_value:.2f}")
ocbc360_value = ocbc360(person1_contribution, person2_contribution, account_balance, years)
print(f"OCBC 360 Account Value after {years} years: S${ocbc360_value:.2f}")

No Interest Account Value after 3 years: S$144000.00
UOB One Account Value after 3 years: S$150195.51
UOB Stash Account Value after 3 years: S$149202.56
OCBC 360 Account Value after 3 years: S$150256.80


# Ultimate Strategy
This is where I will dive into slightly more complex data sceince techniques. I will be using a grid search to find the ideal account methods to invest your money.

Assumptions:
1. The money will only be moved every 12 months (interest is on a p.a basis anyways)
2. Constant contribution from both parties
3. No shocks to the input (Meaning no withdrawal or sudden input other than the monthly contribution)

Below is the function

In [38]:
import itertools
import math

def find_best_annual_strategy(
    person1_monthly_contribution, 
    person2_monthly_contribution, 
    simulation_initial_balance, # Overall starting balance
    total_simulation_years
):
    """
    Compares banking strategies with annual switching points, using pre-defined bank functions.
    OCBC 360 assumes all conditions are met by default as per its function definition.
    """
    if not isinstance(total_simulation_years, int) or total_simulation_years <= 0:
        raise ValueError("Total simulation years must be a positive integer.")

    # Map bank names to their respective functions
    # Note the different initial balance parameter names in your functions
    bank_config = {
        "UOB One": {"func": uob_one, "initial_bal_param_name": "account_balance"},
        "UOB Stash": {"func": uob_stash, "initial_bal_param_name": "initial_account_balance"},
        "OCBC 360": {"func": ocbc360, "initial_bal_param_name": "initial_account_balance"}
    }
    bank_names = list(bank_config.keys())
    all_strategy_results = {}

    # 1. Consistent Strategies (using one bank for all years)
    print("Evaluating Consistent Strategies:")
    for bank_name in bank_names:
        config = bank_config[bank_name]
        bank_func = config["func"]
        initial_bal_param_name = config["initial_bal_param_name"]
        
        kwargs = {
            "person1_monthly_contribution": person1_monthly_contribution,
            "person2_monthly_contribution": person2_monthly_contribution,
            initial_bal_param_name: simulation_initial_balance,
            "years": total_simulation_years
        }
        # For uob_one, the param names are different in its definition
        if bank_name == "UOB One":
            kwargs = {
                "person1_contribution": person1_monthly_contribution, # as per your uob_one signature
                "person2_contribution": person2_monthly_contribution, # as per your uob_one signature
                initial_bal_param_name: simulation_initial_balance,
                "years": total_simulation_years
            }

        final_balance = bank_func(**kwargs)
        strategy_name_key = f"{bank_name} (Consistent)"
        all_strategy_results[strategy_name_key] = round(final_balance, 2)
        print(f"  {strategy_name_key}: S${final_balance:,.2f}")


    # 2. Mixed Strategies (switching annually)
    print("\nEvaluating Mixed Strategies (Annual Switching):")
    if total_simulation_years > 0: # Mixed strategies require at least 1 year
        # Generate all sequences of bank choices, one for each year
        for annual_bank_choice_sequence in itertools.product(bank_names, repeat=total_simulation_years):
            current_balance_for_this_strategy = simulation_initial_balance
            strategy_description_parts = []

            for year_idx in range(total_simulation_years):
                chosen_bank_name_for_year = annual_bank_choice_sequence[year_idx]
                config = bank_config[chosen_bank_name_for_year]
                bank_func_for_year = config["func"]
                initial_bal_param_name_segment = config["initial_bal_param_name"]
                
                strategy_description_parts.append(f"Yr{year_idx+1}:{chosen_bank_name_for_year.replace(' ', '')[:7]}")

                kwargs_segment = {
                    "person1_monthly_contribution": person1_monthly_contribution,
                    "person2_monthly_contribution": person2_monthly_contribution,
                    initial_bal_param_name_segment: current_balance_for_this_strategy,
                    "years": 1 # Simulate for 1 year
                }
                if chosen_bank_name_for_year == "UOB One":
                     kwargs_segment = {
                        "person1_contribution": person1_monthly_contribution,
                        "person2_contribution": person2_monthly_contribution,
                        initial_bal_param_name_segment: current_balance_for_this_strategy,
                        "years": 1
                    }
                
                current_balance_for_this_strategy = bank_func_for_year(**kwargs_segment)
            
            full_strategy_name = "Mixed: (" + " -> ".join(strategy_description_parts) + ")"
            all_strategy_results[full_strategy_name] = round(current_balance_for_this_strategy, 2)
            # Optional: print progress for long simulations
            # if len(all_strategy_results) % 50 == 0:
            # print(f"  Evaluated {len(all_strategy_results) - 3} mixed strategies...")


    # Determine the ideal strategy
    if not all_strategy_results:
        ideal_strategy_name = "No strategies evaluated."
        max_balance = simulation_initial_balance
    else:
        ideal_strategy_name = max(all_strategy_results, key=all_strategy_results.get)
        max_balance = all_strategy_results[ideal_strategy_name]

    return ideal_strategy_name, max_balance, all_strategy_results


This is the implementation of the function, please remember to define your inputs in jupyter notebook.

In [39]:
# --- Define Your Inputs ---
p1_contrib = 1000
p2_contrib = 1000
initial_bal = 10000
years_to_simulate = 5 # Use less than 5 for faster testing, but anything above that will most probably not destroy your computer anyways

# --- Run the Comparison ---
print(f"Starting strategy comparison for {years_to_simulate} years...\n")
best_strategy_name, highest_balance, all_results = find_best_annual_strategy(
    p1_contrib,
    p2_contrib,
    initial_bal,
    years_to_simulate
)

# --- Output Results ---
print(f"\n--- Bank Account Strategy Comparison Results ---")
print(f"Simulation Period: {years_to_simulate} years")
print(f"Initial Balance: S${initial_bal:,.2f}")
print(f"Person 1 Monthly Contribution: S${p1_contrib:,.2f}")
print(f"Person 2 Monthly Contribution: S${p2_contrib:,.2f}\n")

print("Results for all tested strategies (Top results shown if many):")
# Sort results for better readability
sorted_results = sorted(all_results.items(), key=lambda item: item[1], reverse=True)

for i, (strategy, balance) in enumerate(sorted_results):
    print(f"  - Strategy: {strategy:<60} | Final Balance: S${balance:,.2f}")
    if i >= 20 and len(sorted_results) > 22: # Limit printing if too many strategies
        print(f"  ... and {len(sorted_results) - (i+1)} more ...")
        break


print(f"\nIdeal Strategy Found Among Tested: {best_strategy_name}")
print(f"Resulting Maximum Balance: S${highest_balance:,.2f}")

Starting strategy comparison for 5 years...

Evaluating Consistent Strategies:
  UOB One (Consistent): S$139,768.78
  UOB Stash (Consistent): S$138,619.13
  OCBC 360 (Consistent): S$140,512.38

Evaluating Mixed Strategies (Annual Switching):

--- Bank Account Strategy Comparison Results ---
Simulation Period: 5 years
Initial Balance: S$10,000.00
Person 1 Monthly Contribution: S$1,000.00
Person 2 Monthly Contribution: S$1,000.00

Results for all tested strategies (Top results shown if many):
  - Strategy: Mixed: (Yr1:OCBC360 -> Yr2:OCBC360 -> Yr3:OCBC360 -> Yr4:OCBC360 -> Yr5:UOBOne) | Final Balance: S$141,006.51
  - Strategy: Mixed: (Yr1:UOBOne -> Yr2:OCBC360 -> Yr3:OCBC360 -> Yr4:OCBC360 -> Yr5:UOBOne) | Final Balance: S$140,885.73
  - Strategy: Mixed: (Yr1:OCBC360 -> Yr2:UOBOne -> Yr3:OCBC360 -> Yr4:OCBC360 -> Yr5:UOBOne) | Final Balance: S$140,762.56
  - Strategy: Mixed: (Yr1:OCBC360 -> Yr2:OCBC360 -> Yr3:OCBC360 -> Yr4:UOBStas -> Yr5:UOBOne) | Final Balance: S$140,694.06
  - Strate

# Final take
The data is clear, for bank accounts less than \$100,000, the OCBC 360 provides the best interest rates. However, for values more than \$100,000, the UOB One is most suitable.

However, the data does not explain the complexity of choosing cards. OCBC Credit Cards, which is required for the spending requirement, are not the most ideal cards in the credit card industry. On the other hand, UOB One cards are known for their somewhat generous miles and cashback. This information cannot be captured through this data analysis. Some might find the petrol cashback of the OCBC 360 to be more suitable for their needs, which others love the high miles to cash ratio of UOB. 